In [ ]:
import os
from os.path import join
from io import StringIO
from textwrap import dedent

import sympy
from sympy import Symbol, sqrt, cos, pi, symbols
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from collections import defaultdict

import qnet
from qnet.algebra import *

import QDYN

from src.notebook_plots_v1 import plot_bs_decay, display_hamiltonian, display_eq, show_summary_dicke
from src.single_sided_network_v1 import network_slh
from src.dicke_single_model_v1 import write_dicke_single_model, err_dicke_single

from doit.tools import register_doit_as_IPython_magic
import clusterjob

In [ ]:
qnet.init_printing()

In [ ]:
register_doit_as_IPython_magic()

In [ ]:
#clusterjob.JobScript.read_defaults('./config/copper.ini')

In [ ]:
clusterjob.JobScript.read_defaults('./config/mlhpc_cluster.ini')

$
\newcommand{ket}[1]{\vert #1 \rangle}
\newcommand{bra}[1]{\langle #1 \vert}
\newcommand{Op}[1]{\hat{#1}}
$

# QSL for Single-Excitation Dicke state

## action wrappers

In [ ]:
def update_config(rf, lambda_a, iter_stop):
    config = QDYN.config.read_config_file(join(rf, 'config'))
    config['oct']['iter_stop'] = iter_stop
    for pulse_config in config['pulse']:
        pulse_config['oct_lambda_a'] = lambda_a
    QDYN.config.write_config(config, join(rf, 'config'))

In [ ]:
def submit_optimization(rf, n_trajs, task):
    """Asynchronously run optimization"""
    if os.path.isfile(join(rf, 'oct.job.dump')):
        return
    body = dedent(r'''
    {module_load}

    cd {rf}
    OMP_NUM_THREADS=1 {runner} qdyn_optimize --n-trajs={n_trajs} \
         --J_T=J_T_sm .
    ''')
    taskname = "oct_%s" % task.name.replace(":", '_')
    jobscript = clusterjob.JobScript(
        body=body, filename=join(rf, 'oct.slr'),
        jobname=taskname, nodes=1, ppn=int(n_trajs), threads=1,
        stdout=join(rf, 'oct.log'))
    jobscript.rf = rf
    runner = {
        'copper': 'aprun -B',
        'mlhpc': 'mpirun -n %d' % n_trajs}
    jobscript.runner = runner[jobscript.cluster_name]
    jobscript.n_trajs = str(int(n_trajs))
    run = jobscript.submit(cache_id=taskname)
    run.dump(join(rf, 'oct.job.dump'))

In [ ]:
def wait_for_clusterjob(dumpfile):
    """Wait until the clusterjob.AsyncResult cached in the given dumpfile ends"""
    try:
        run = clusterjob.AsyncResult.load(dumpfile)
        run.wait()
        os.unlink(dumpfile)
        return run.successful()
    except OSError:
        pass

## custom uptodate routines

In [ ]:
from src.qdyn_model_v1 import pulses_uptodate

## task definitions

In [ ]:
def runfolder(row):
    return './data/dicke1_QSL/rf_%dnodes_T%d' % (row['n_nodes'], row['T'])

In [ ]:
def task_create_runfolder():
    """Create all necessary runfolders for the runs defined in params_df"""
    jobs = {}
    slh_models = {}
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        n_nodes = int(row['n_nodes'])
        try:
            slh = slh_models[n_nodes]
        except KeyError:
            slh = network_slh(
                n_cavity=2, n_nodes=n_nodes, topology='driven_bs_fb')
            slh_models[n_nodes] = slh
        if n_nodes not in slh_models:
            slh_mode
        if rf in jobs:
            continue
        jobs[rf] = {
            'name': str(rf),
            'actions': [
                (write_dicke_single_model, [slh, ], dict(
                    rf=rf, T=row['T'], theta=0, nt=500,
                    kappa=1.0, E0_cycles=2, mcwf=True, non_herm=True,
                    lambda_a=row['lambda_a'],
                    iter_stop=int(row['iter_stop'])))],
            'targets': [join(rf, 'config')],
            'uptodate': [True, ] # up to date if target exists
        }
    for job in jobs.values():
        yield job

In [ ]:
def task_update_runfolder():
    """For every row in params_df, update the config file in the appropriate
    runfolder with the value in that row"""
    rf_jobs = defaultdict(list)
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        # we only update the config after any earlier optimization has finished
        task_dep = ['wait_for_optimization:%s' % ind2 for ind2 in rf_jobs[rf]]
        rf_jobs[rf].append(ind)
        yield {
            'name': str(ind),
            'actions': [
                (update_config, [], dict(
                    rf=rf, lambda_a=row['lambda_a'],
                    iter_stop=int(row['iter_stop'])))],
            'file_dep': [join(rf, 'config')],
            'uptodate': [False, ],  # always run task
            'task_dep': task_dep}


In [ ]:
def task_submit_optimization():
    """Run optimization for every runfolder from params_df"""
    rf_jobs = defaultdict(list)
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        task_dep = ['wait_for_optimization:%s' % ind2 for ind2 in rf_jobs[rf]]
        task_dep.append('update_runfolder:%s' % ind)
        yield {
            'name': str(ind),
            'actions': [
                (submit_optimization, [rf, ], dict(n_trajs=row['n_trajs']))],
                # 'task' keyword arg is added automatically
            'task_dep': task_dep,
            'uptodate': [(pulses_uptodate, [], {'rf': rf}), ],
        }

In [ ]:
def task_wait_for_optimization():
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        yield {
            'name': str(ind),
            'task_dep': ['submit_optimization:%d' % ind],
            'actions': [
                (wait_for_clusterjob, [join(rf, 'oct.job.dump')], {}),]}

##  OCT Submission

In [ ]:
params_data_str = r'''
# n_nodes   T  lambda_a  n_trajs   iter_stop
        2  10     0.001       10        1000
        2  20     0.001       10        1000
        2  50     0.001       10        1000
        2  70     0.001       10        1000
        3  10     0.001       10        1000
        3  20     0.001       10        1000
        3  50     0.001       10        1000
        3  70     0.001       10        1000
        4  10     0.001       10        1000
        4  20     0.001       10        1000
        4  50     0.001       10        1000
        4  70     0.001       10        1000
        5  10     0.001       10        1000
        5  20     0.001       10        1000
        5  50     0.001       10        1000
        5  70     0.001       10        1000
        6  10     0.001       10        1000
        6  20     0.001       10        1000
        6  50     0.001       10        1000
        6  70     0.001       10        1000
'''
params_df = pd.read_fwf(
        StringIO(params_data_str), comment='#', header=1,
        names=['n_nodes', 'T', 'lambda_a', 'n_trajs', 'iter_stop'],
        converters={'n_nodes': int, 'T': int, 'lambda_a': float,
                    'n_trajs': int, 'iter_stop': int})

In [ ]:
# DEBUG
#
#
params_data_str = r'''
# n_nodes   T  lambda_a  n_trajs   iter_stop
        2  10     0.001       10        1000
        2  20     0.001       10        1000
'''
params_df = pd.read_fwf(
        StringIO(params_data_str), comment='#', header=1,
        names=['n_nodes', 'T', 'lambda_a', 'n_trajs', 'iter_stop'],
        converters={'n_nodes': int, 'T': int, 'lambda_a': float,
                    'n_trajs': int, 'iter_stop': int})

In [ ]:
params_df

In [ ]:
import logging
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(level=logging.DEBUG, filename='./data/dicke1_QSL_clusterjob.log')

In [ ]:
%doit -n 20 wait_for_optimization